# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [ ]:
import pandas as pd 
from pyspark.sql import SparkSession

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [ ]:
spark = SparkSession.builder \
    .appName("Leer CSV") \
    .getOrCreate()

### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [ ]:
file_path = "C:\Users\maria\Documents\ITAM\Materias\7° Semestre\Fuentes/datosTarea.csv"

#DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show()

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [ ]:
filtered_df = df.filter(df.Name.startswith("M") & (df["Number of employees"] >= 4000) & (df["Number of employees"] <= 7000))

# Seleccionar solo las columnas 'Name' y 'Number of employees'
selected_columns_df = filtered_df.select("Name", "Number of employees")
selected_columns_df.show()

>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [ ]:
from pyspark.sql.functions import col

filtered_df = df.filter(~col("Country").startswith("B") &
                        ~col("Country").startswith("S") &
                        ~col("Country").startswith("M") &
                        (col("Networth") <= 500000))

selected_columns_df = filtered_df.select("Name", "Country", "Networth")
selected_columns_df.show()

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd

# pandas_udf
@pandas_udf("double", PandasUDFType.SCALAR)
def adjust_profit(profit_series):
    return profit_series - profit_series.mean()

# DataFrame y una nueva columna
adjusted_df = df.withColumn("Adjusted Profit", adjust_profit(df["profit"]))
adjusted_df.show()

### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Ventana de partición
windowSpec = Window.partitionBy("Industry").orderBy(df["profit"].desc())

top3_df = df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter("row_number <= 3") \
    .select("Industry", "Name", "profit", "row_number")
top3_df.show()

>Agrupa por industry y calcula el promedio de empleados que tienen

In [ ]:
average_df = df.groupBy("Industry").agg(avg("Number of employees").alias("Average Employees"))
average_df.show()

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [ ]:
df.createOrReplaceTempView("companies")

# Ejecutar la consulta SQL para contar las empresas fundadas después del 2000
result = spark.sql("SELECT COUNT(*) as TotalCompanies FROM companies WHERE Founded > 2000")

# Mostrar el resultado
result.show()

# Detener la SparkSession
spark.stop()

### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# VectorAssembler
assembler = VectorAssembler(inputCols=["Number of employees", "Networth", "stock_price"], outputCol="features")
data = assembler.transform(df)
final_data = data.select("features", "profit")

# regresión lineal
lr_model = LinearRegression(featuresCol="features", labelCol="profit")
train_data, test_data = final_data.randomSplit([0.7, 0.3])
trained_model = lr_model.fit(train_data)
predictions = trained_model.transform(test_data)
predictions.select("prediction", "profit").show()

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [ ]:
predictions_pandas = predictions.to_pandas_on_spark()
output_path = "C:\Users\maria\Documents\ITAM\Materias\7° Semestre\Fuentes/outputDatosTarea.csv"
predictions_pandas.to_csv(output_path)

# Detener la SparkSession
spark.stop()